In [51]:
# !pip install git+https://github.com/ytarazona/scikit-eo

  Cloning https://github.com/ytarazona/scikit-eo to c:\users\wenxinyang\appdata\local\temp\pip-req-build-dw_hlqz0
  Resolved https://github.com/ytarazona/scikit-eo to commit eb0a6cdca8369058d446428618a1677dcbe553a7
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for scikeo: filename=scikeo-0.2.16-py2.py3-none-any.whl size=37729 sha256=9d78417de6e2b4f8ae1967df306062c5b38f9e05c8b2d054fd0b8d3db28570e7
  Stored in directory: C:\Users\wenxinyang\AppData\Local\Temp\pip-ephem-wheel-cache-u82omjxc\wheels\1b\3d\0a\263fb76deccb8f6c6995a90e117116060f455fb73e3855a67b
Successfully built scikeo
  Attempting uninstall: scikeo
    Found existing installation: scikeo 0.2.32
    Uninstalling scikeo-0.2.32:
      Successfully uninstalled scikeo-0.2.32


  Running command git clone --filter=blob:none --quiet https://github.com/ytarazona/scikit-eo 'C:\Users\wenxinyang\AppData\Local\Temp\pip-req-build-dw_hlqz0'


In [1]:
import rasterio
import numpy as np
from scikeo.mla import MLA
import matplotlib.pyplot as plt
from dbfread import DBF
import matplotlib as mpl
import pandas as pd
from scikeo.process import extract
from scikeo.deeplearning import DL

In [2]:
import geopandas as gpd

In [3]:
from shapely.geometry import Polygon

In [4]:
path_raster = r"C:\Users\wenxinyang\Desktop\Projects\iguide\2020_houston_naip.tif"
img = rasterio.open(path_raster)

In [5]:
img.crs

CRS.from_epsg(4326)

In [6]:
path_endm = r"C:\Users\wenxinyang\Desktop\Projects\iguide\park_hst\park_hst.dbf"
endm = DBF(path_endm)

In [7]:
test = gpd.read_file(r"C:\Users\wenxinyang\Desktop\Projects\iguide\park_hst\park_hst.shp")

In [8]:
test_pt = gpd.GeoDataFrame(test.sample_points(size = 30))

In [9]:
test_pt['geometry'] = test_pt['sampled_points']

C:\Users\wenxinyang\AppData\Local\Temp\ipykernel_6716\3156293003.py:1: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  test_pt['geometry'] = test_pt['sampled_points']


In [10]:
test_pt = test_pt.set_geometry('geometry').explode().reset_index()

C:\Users\wenxinyang\AppData\Local\Temp\ipykernel_6716\748659652.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  test_pt = test_pt.set_geometry('geometry').explode().reset_index()


In [11]:
test_pt.columns

Index(['level_0', 'level_1', 'sampled_points', 'geometry'], dtype='object')

In [12]:
test_pt['class'] = 1

In [42]:
# used its extract function

In [13]:
gdf = gpd.GeoDataFrame(geometry = test_pt['geometry'])
coords = [(x,y) for x,y in zip(gdf['geometry'].x, gdf['geometry'].y)]

In [14]:
gdf['values'] = [x for x in img.sample(coords)]

In [15]:
gdf

,geometry,values
0,POINT (-95.58503 29.96638),"[94.5, 111.0, 92.5, 159.0]"
1,POINT (-95.58474 29.96662),"[77.0, 95.0, 83.0, 150.5]"
2,POINT (-95.58470 29.96645),"[77.0, 95.0, 83.0, 150.5]"
3,POINT (-95.58455 29.96546),"[128.5, 138.0, 121.5, 140.0]"
4,POINT (-95.58440 29.96666),"[108.0, 123.5, 101.5, 152.5]"
...,...,...
70315,POINT (-95.71444 29.60745),"[128.0, 139.5, 104.0, 178.0]"
70316,POINT (-95.71431 29.60794),"[127.5, 140.0, 106.5, 174.0]"
70317,POINT (-95.71422 29.60715),"[171.5, 173.5, 164.0, 131.0]"
70318,POINT (-95.71418 29.60698),"[171.5, 173.5, 164.0, 131.0]"


In [16]:
bands_total = gdf['values'][0].shape[0]

In [17]:
col_names = [f'band{i}' for i in range(1, bands_total+1)]

In [18]:
col_names

['band1', 'band2', 'band3', 'band4']

In [19]:
df = pd.DataFrame(gdf['values'].to_list(), columns = col_names)

In [20]:
df

,band1,band2,band3,band4
0,94.5,111.0,92.5,159.0
1,77.0,95.0,83.0,150.5
2,77.0,95.0,83.0,150.5
3,128.5,138.0,121.5,140.0
4,108.0,123.5,101.5,152.5
...,...,...,...,...
70315,128.0,139.5,104.0,178.0
70316,127.5,140.0,106.5,174.0
70317,171.5,173.5,164.0,131.0
70318,171.5,173.5,164.0,131.0


In [21]:
join_df = pd.concat([test_pt.iloc[:,0], df], axis=1, join='inner')

In [22]:
join_df

,level_0,band1,band2,band3,band4
0,0,94.5,111.0,92.5,159.0
1,0,77.0,95.0,83.0,150.5
2,0,77.0,95.0,83.0,150.5
3,0,128.5,138.0,121.5,140.0
4,0,108.0,123.5,101.5,152.5
...,...,...,...,...,...
70315,2343,128.0,139.5,104.0,178.0
70316,2343,127.5,140.0,106.5,174.0
70317,2343,171.5,173.5,164.0,131.0
70318,2343,171.5,173.5,164.0,131.0


In [23]:
join_df_valid = join_df.dropna(how='any',axis=0) 

In [24]:
join_df_valid = join_df_valid.drop(['level_0'], axis=1)

In [25]:
join_df_valid['class'] = 1

In [26]:
join_df_valid.isnull().values.any()

False

In [27]:
inst = DL(image = img, endmembers = join_df_valid)

In [29]:
fc = inst.FullyConnected(hidden_layers = 2,
                         hidden_units = [64, 16, 8, 8],
                         output_units = 4,
                         input_shape = (2,),
                         epochs = 2,
                         batch_size = 32,
                         training_split = 0.8)

SyntaxError: invalid syntax (1221396191.py, line 4)